In [2]:
import re
import os
import pandas as pd
import ngrams
import numpy as np
import math
import csv
import fasttext

In [3]:
#全局变量
attributeList = []
#token的字典，用来记录token出现的次数nt
tokenDict={}

In [5]:
#将属性名按照标点，分割成token
def snake_case_split(line):
    #Python strip() 方法用于移除字符串头尾指定的字符（默认为空格或换行符）或字符序列。
    #r表示非转义的原始字符串
    #由于正则表达式通常都包含反斜杠，所以你最好使用原始字符串来表示它们。模式元素(如 r'\t'，等价于 '\\t')匹配相应的特殊字符。
    #[...]用来表示一组字符,单独列出：[amk] 匹配 'a'，'m'或'k'
    line_split = re.split(r'[\s_]',line.strip())
    line_split = [line.strip() for line in line_split if len(line.strip())>0]
    return line_split




#将属性按照大写字母，分割成token
#"firstName"分成 ['First', 'Name']
def camel_case_split(str):
    if "a"<=str[0]<="z" :
        strTemp=str[0].upper()+str[1:]
        return re.findall(r'[A-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))', strTemp)
    else :
        return re.findall(r'[A-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))', str)




#将属性名先按照snakeCase拆分，再按照camelCase拆分
def attributeToToken(str):
    tokenList = []
    for snakeToken in snake_case_split(str):
        for camelToken in camel_case_split(snakeToken):
            camelToken = camelToken.lower()
            corpusTokenList = ngrams.segment2(camelToken) [1]
            tokenList = tokenList + corpusTokenList
            for finalToken in corpusTokenList:                
                #生成token的时候就更新tokenDict
                if finalToken in tokenDict:
                    tokenDict[finalToken] = tokenDict[finalToken]+1
                else:
                    tokenDict[finalToken] = 1
    return tokenList

In [6]:
class Attribute:
    def __init__(self, attributeName, originateFrom):
        self.attributeName = attributeName
        self.tokenList = attributeToToken(attributeName)
        #属性的词嵌入表示
        #加权平均
        #权和N有关
        #numOfAttribute得读完整个数据湖才知道
        #self.wordEmbedding = tokenToVector(self.tokenList)
        
        #属性来自哪张表
        self.originateFrom = originateFrom

In [ ]:
class Table:
    def __init__(self, ):

In [7]:
#getTokenWeight函数用到numOfAttribute
numOfAttribute = 0
numOfTable = 0

#将湖中所有属性读到attributeList中
#for循环中的info代表文件名
for info in os.listdir('/usr/dataLake/datasets/BenchmarkCsvfiles'): 
    #去掉.csv后缀
    tableName = info[0:-4]
    domain = os.path.abspath('/usr/dataLake/datasets/BenchmarkCsvfiles') #获取文件夹的路径
    info = os.path.join(domain,info) #将路径与文件名结合起来就是每个文件的完整路径
    print(info)
    df = pd.read_csv(info)
    for attributeName in list(df):
        #Attribute类的构造函数会更新tokenDict
        x=Attribute(attributeName, tableName)
        numOfAttribute = numOfAttribute+1
        #通过append函数将自定义类的对象放入list中
        attributeList.append(x)
        #print(attributeList[i])可证明是Attribute类的对象
    
    numOfTable = numOfTable + 1
    if numOfTable == 10:
        break

/usr/dataLake/datasets/BenchmarkCsvfiles/t_93f3d6f7fc6aa6ff____c13_0____0.csv
/usr/dataLake/datasets/BenchmarkCsvfiles/t_70941cace7dd1c45____c12_1____1.csv
/usr/dataLake/datasets/BenchmarkCsvfiles/t_e9efd5cda78af711____c4_1____1.csv
/usr/dataLake/datasets/BenchmarkCsvfiles/t_ca85e8f9eef5b9d5____c5_1____0.csv
/usr/dataLake/datasets/BenchmarkCsvfiles/t_dc9442ed0b52d69c____c13_0____4.csv
/usr/dataLake/datasets/BenchmarkCsvfiles/PED_SK_DTL_SNF____c6_0____4.csv
/usr/dataLake/datasets/BenchmarkCsvfiles/t_93f3d6f7fc6aa6ff____c21_0____4.csv
/usr/dataLake/datasets/BenchmarkCsvfiles/CTA_Ridership_Avg_Weekday_Bus_Stop_Boardings_in_October_2012____c7_0____2.csv
/usr/dataLake/datasets/BenchmarkCsvfiles/t_014c2259cce4ab13____c25_0____4.csv
/usr/dataLake/datasets/BenchmarkCsvfiles/CTA_Ridership_Avg_Weekday_Bus_Stop_Boardings_in_October_2012____c2_1____3.csv


In [7]:
print(tokenDict)

{'address': 2, 'auth': 2, 'description': 4, 'granted': 2, 'date': 9, 'lic': 2, 'no': 5, 'pub': 2, 'text': 2, 'reg': 7, 'service': 6, 'number': 2, 'type': 9, 'other': 2, 'details': 3, 'tao': 2, 'covered': 2, 'by': 2, 'area': 2, 'effective': 2, 'end': 2, 'trading': 2, 'name': 4, 'quality': 25, 'indicator': 25, 'indicateurs': 29, 'de': 29, 'qualit': 29, 'kind': 1, 'pmc': 1, 'he': 1, 'pm': 4, 'te': 4, 's': 3, 'grou': 1, 'colour': 1, 'consis': 1, 'pls': 1, 'mottle': 1, 'frq': 1, 'root': 2, 'freq': 1, 'sec': 1, 'grad': 1, 'rest': 1, 'ri': 1, 'soil': 1, 'group': 1, 'exp': 1, 'op': 1, 'registration': 1, 'status': 1, 'subsidies': 1, 'finish': 1, 'point': 2, 'received': 1, 'start': 1, 'via': 1, 'a': 1, 'lightings': 1, 'boardings': 1, 'cross': 1, 'street': 3, 'location': 2, 'on': 2, 'routes': 1, 'stop': 2, 'id': 2, 'aircraft': 6, 'make': 4, 'airport': 1, 'icao': 1, 'incident': 1, 'occ': 2, 'ocean': 1, 'operator': 2, 'organization': 5, 'country': 5}


In [8]:
#输入target表
targetTable='t_93f3d6f7fc6aa6ff____c13_0____0'
targetAttributeList = []
info = targetTable+'.csv'
domain = os.path.abspath('/usr/dataLake/datasets/BenchmarkCsvfiles') #获取文件夹的路径
info = os.path.join(domain,info) #将路径与文件名结合起来就是每个文件的完整路径
df = pd.read_csv(info)
for targetAttributeName in list(df):
    #targetAttribute的token被额外加了一次
    targetAttribute=Attribute(targetAttributeName, targetTable)
    targetAttributeList.append(targetAttribute)
    for token in targetAttribute.tokenList:
        #矫正tokendict
        #已经通过验证
        tokenDict[token] = tokenDict[token]-1

In [9]:
print(tokenDict)

{'address': 2, 'auth': 2, 'description': 4, 'granted': 2, 'date': 9, 'lic': 2, 'no': 5, 'pub': 2, 'text': 2, 'reg': 7, 'service': 6, 'number': 2, 'type': 9, 'other': 2, 'details': 3, 'tao': 2, 'covered': 2, 'by': 2, 'area': 2, 'effective': 2, 'end': 2, 'trading': 2, 'name': 4, 'quality': 25, 'indicator': 25, 'indicateurs': 29, 'de': 29, 'qualit': 29, 'kind': 1, 'pmc': 1, 'he': 1, 'pm': 4, 'te': 4, 's': 3, 'grou': 1, 'colour': 1, 'consis': 1, 'pls': 1, 'mottle': 1, 'frq': 1, 'root': 2, 'freq': 1, 'sec': 1, 'grad': 1, 'rest': 1, 'ri': 1, 'soil': 1, 'group': 1, 'exp': 1, 'op': 1, 'registration': 1, 'status': 1, 'subsidies': 1, 'finish': 1, 'point': 2, 'received': 1, 'start': 1, 'via': 1, 'a': 1, 'lightings': 1, 'boardings': 1, 'cross': 1, 'street': 3, 'location': 2, 'on': 2, 'routes': 1, 'stop': 2, 'id': 2, 'aircraft': 6, 'make': 4, 'airport': 1, 'icao': 1, 'incident': 1, 'occ': 2, 'ocean': 1, 'operator': 2, 'organization': 5, 'country': 5}


In [9]:
#加载模型

fasttext.FastText.eprint = lambda x: None
model = fasttext.load_model("/usr/dataLake/result/fil9.bin")

def getTokenWeight(token):
    nt = tokenDict[token]
    #默认以e为底
    return math.log(numOfAttribute/nt)

#归一化向量
def normalize(x):
    return x/np.linalg.norm(x)

#将属性名的tokenList转换为词嵌入向量
#获取token对应的词嵌入向量，归一化。加权平均后，再归一化一下。
def tokenToVector(tokenList):
    #获取词嵌入
    wordEmbedding = model.get_word_vector(tokenList[0])
    #函数返回值
    unitVector = getTokenWeight(tokenList[0]) * normalize(wordEmbedding)
    #加权平均的分母
    totalWeight = getTokenWeight(tokenList[0])
    for token in tokenList[1:]:
        wordEmbedding = model.get_word_vector(token)
        unitVector = unitVector + getTokenWeight(token) * normalize(wordEmbedding)
        totalWeight = totalWeight+getTokenWeight(token)    
    return normalize(unitVector/totalWeight)


In [10]:
#计算attributeVector，并输出到文件中
with open('attributeVector.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    for targetAttribute in targetAttributeList:
        targetAttribute.wordEmbedding = tokenToVector(targetAttribute.tokenList)
        writer.writerow(targetAttribute.wordEmbedding)

    for attribute in attributeList:
        attribute.wordEmbedding = tokenToVector(attribute.tokenList)
        writer.writerow(attribute.wordEmbedding)

In [20]:
for i in range(13,25):
    print(attributeList[i].attributeName)

Quality Indicator
 Indicateurs de qualit�
Quality Indicator
 Indicateurs de qualit� Indicateurs de qualit�
Quality Indicator
 Indicateurs de qualit� Indicateurs de qualit�.1
Quality Indicator
Indicateurs de qualit�
Quality Indicator
Indicateurs de qualit�.2
Quality Indicator
Indicateurs de qualit�.3
Quality Indicator
Indicateurs de qualit�.4
Quality Indicator
Indicateurs de qualit�.5
Quality Indicator
Indicateurs de qualit�.6
Quality Indicator
Indicateurs de qualit�.7
Quality Indicator
Indicateurs de qualit�.8
Quality Indicator
Indicateurs de qualit�.9


In [24]:
#用list存储列的类别信息
ClusteringResults=[]
with open('/usr/dataLake/DRD/ball-k-means/ClusteringResults.csv', 'r') as f:
    for line in f:
        ClusteringResults.append(int(line))

#已经验证ClusteringResults的长度
#元素的类型

121

In [ ]:
k=40
CandidateTablePair={}
#直接把数据读到Clust中，等价转换
ClusterPointIndex=[]


In [37]:
def read_csv(file_name):
    f = open(file_name, 'r')
    content = f.read()
    final_list = list()
    rows = content.split('\n')
    for row in rows:
        final_list.append(row.split(','))
    return final_list

#二维list
#type已经验证
#二维list数据正确性已经验证
#-1是为了删除末尾的空列表
ClusterPointIndex = read_csv("/usr/dataLake/DRD/ball-k-means/ClusterPointIndex.csv")[0:-1]
print(ClusterPointIndex)

[['94', '95', '96', '97'], ['105'], ['4', '17', '72'], ['27', '28', '48', '49'], ['8', '21', '77'], ['90', '103'], ['114', '115', '116', '117', '118'], ['3', '16', '70'], ['63'], ['10', '23', '80'], ['109', '110', '111', '112', '113'], ['41', '71'], ['64'], ['42', '44'], ['5', '18', '73'], ['40', '60', '61', '62'], ['0', '13', '66'], ['88', '91', '119'], ['45'], ['9', '22', '79'], ['89'], ['1', '14', '67'], ['39', '65'], ['26', '29', '30', '31', '32', '33', '34', '35', '36', '37', '47', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59'], ['92'], ['86'], ['98', '99', '107', '108'], ['93', '120'], ['2', '15', '69', '74', '83'], ['100', '101'], ['104'], ['12', '25', '85'], ['78'], ['46', '68'], ['11', '24', '81'], ['6', '19', '75'], ['106'], ['38'], ['7', '20', '43', '76', '87', '102'], ['82', '84']]


In [ ]:
#nt小，ITF大，权重大。
#token越稀有，权重越大。
#N是数据湖中有多少列。